In [45]:
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold

import numpy as np
from scipy import sparse

### Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [2]:
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

In [3]:
text = newsgroups.data
y = newsgroups.target

### Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [13]:
vectorizer = TfidfVectorizer()

In [15]:
X = vectorizer.fit_transform(text)

In [22]:
# feature_mapping = vectorizer.get_feature_names()
# print (feature_mapping)

### Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [25]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}

In [28]:
cv = KFold(n_splits=5, shuffle=True, random_state=241)

In [29]:
clf = SVC(kernel='linear', random_state=241)

In [30]:
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)

In [31]:
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

### Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге

In [34]:
print('best score: ', gs.best_score_)

best score:  0.993281075028


In [35]:
print('best param: ', gs.best_params_)

best param:  {'C': 1.0}


### Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [36]:
best_estimator = gs.best_estimator_

### Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.


In [108]:
coef = sparse.csr_matrix(best_estimator.coef_, dtype=np.float32).toarray()[0]

In [110]:
coef = np.abs(coef)

In [111]:
indexes = []
for i in range(10):
    amax = coef.argmax()
    indexes.append(amax)
    coef[amax] = 0

In [112]:
feature_mapping = vectorizer.get_feature_names()

In [113]:
words = []
for i in indexes:
    words.append(str(feature_mapping[i]).lower())

In [114]:
with open('output.txt', 'w') as f:
    f.write(' '.join(np.sort(words)))